In [3]:
import os
import sys
import logging

import numpy as np
import pandas as pd 

from gensim.models import Word2Vec

import config
from ingredient_parser import ingredient_parser

In [4]:
import pandas as pd
import nltk
import string
import ast
import re
import unidecode
import nltk.corpus as corpus

#nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter
#import config

In [ ]:
sys.path.append('..')

if sys.platform == 'linux':
    path = config.LINUX_PATH
else:
    path = config.OS_PATH
os.chdir(path)

In [5]:
sys.path.append('..')
path = config.OS_PATH

In [3]:
data = pd.read_csv('input/df_recipes.csv')
data['parsed'] = data.ingredients.apply(ingredient_parser)
data.head()

,name,steps,ingredients,ingredients_parsed,parsed
0,arriba baked winter squash mexican style,"['make a choice and proceed with recipe', 'dep...","['winter squash', 'mexican seasoning', 'mixed ...",winter squash mexican seasoning mixed spice ho...,"[winter squash, mexican seasoning, mixed spice..."
1,a bit different breakfast pizza,"['preheat oven to 425 degrees f', 'press dough...","['prepared pizza crust', 'sausage patty', 'egg...",prepared pizza crust sausage patty egg milk sa...,"[prepared pizza crust, sausage patty, eggs, mi..."
2,beat this banana bread,"['preheat oven to 350 degrees', 'butter two 9x...","['sugar', 'unsalted butter', 'bananas', 'eggs'...",sugar unsalted butter banana egg fresh lemon j...,"[sugar, unsalted butter, bananas, eggs, fresh ..."
3,better than sex strawberries,['crush vanilla wafers into fine crumbs and li...,"['vanilla wafers', 'butter', 'powdered sugar',...",vanilla wafer butter powdered sugar egg whippi...,"[vanilla wafers, butter, powdered sugar, eggs,..."
4,better then bush s baked beans,['in a very large sauce pan cover the beans an...,"['great northern bean', 'chicken bouillon cube...",great northern bean chicken bouillon cube dark...,"[great northern bean, chicken bouillon cubes, ..."


In [4]:
data['parsed'].iloc[0]

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [22]:
data['parsed'].dtype

dtype('O')

In [6]:
# get corpus with the documents sorted in alphabetical order
def get_and_sort_corpus(data):
    corpus_sorted = []
    for doc in data.parsed.values:
        doc.sort()
        corpus_sorted.append(doc)
    return corpus_sorted

corpus = get_and_sort_corpus(data)
print(f"Length of corpus: {len(corpus)}")

Length of corpus: 94564


In [7]:
# calculate average length of each document 
lengths = [len(doc) for doc in corpus]
avg_len = float(sum(lengths)) / len(lengths)
avg_len

8.926790321898396

In [58]:
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 6 # number of CPUs
window = 9 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow = Word2Vec(corpus, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100)

In [6]:
model_cbow = Word2Vec.load('models/model_cbow.bin')

In [10]:
#Summarize the loaded model
print(model_cbow)

#Summarize vocabulary
words = list(model_cbow.wv.index_to_key)
words.sort()
# print(words)


Word2Vec(vocab=9868, vector_size=100, alpha=0.025)


In [12]:
#Acess vector for one word
print(model_cbow.wv['cheese'])

[-0.07361066 -1.8324972   1.1452914   0.34367687  0.2713495  -0.3425753
 -0.58229494  0.27240643  1.1556298   0.01240388 -0.8318319  -1.0165732
  0.77112335  0.12960795 -0.9716633   0.17690742  0.8923252  -1.3945265
  0.290436   -0.04339992  0.7580615   1.1052206   0.8737823   1.5322433
 -0.0235903  -0.7472721   0.17979541  0.75999284 -1.6583536  -0.23552205
  1.4295099   1.3505839   1.1024392  -1.0483865   0.49111792  0.7330313
  0.687744   -2.5379298   0.14389125 -0.5970398  -0.42096147 -0.07966862
  0.7793348  -0.1823584   0.7442278   0.3127837  -0.00825099  0.5734534
  1.024176    0.08931772  0.80366546  1.3342966   0.43076077 -1.2120396
 -0.8316845   0.9720223  -0.08080867  1.8078634  -0.14822097  0.41132224
  0.06406178  1.6198484   0.39591515  0.24146229  0.162092    0.6233726
 -1.106728   -0.86356413  0.4644808   0.54935205  0.02982539 -0.00607769
 -0.28322443  0.8798804   1.063792   -1.1455724   0.6673078  -1.7586339
  0.26613596  1.4409537   0.9870299   0.87014985 -0.377703  

In [14]:
# most similar
model_cbow.wv.most_similar(u'cheese')
#model_cbow.wv.similarity('cheese', 'margarine')

[('cheddar cheese', 0.9109072685241699),
 ('colby cheese', 0.762747585773468),
 ('american cheese', 0.7504609227180481),
 ('colby monterey jack cheese', 0.7120832800865173),
 ('mild cheddar cheese', 0.6971008777618408),
 ('chihuahua cheese', 0.6925839781761169),
 ('low fat cheddar cheese', 0.6763399243354797),
 ('corn', 0.6734391450881958),
 ('cottage cheese', 0.6602693796157837),
 ('black olive', 0.6591867208480835)]

In [60]:
model_cbow.save('models/model_cbow.bin')

In [7]:
from word2vec_rec import get_recs

In [ ]:
if __name__ == "__main__":
    # test
    input = "chicken thigh, onion, rice noodle, seaweed nori sheet, sesame, shallot, soy, spinach, star, tofu"
    rec = get_recs(input)
    print(rec)

In [8]:
input = "chicken thigh, onion, rice noodle, seaweed nori sheet, sesame, shallot, soy, spinach, star, tofu"
rec = get_recs(input)
print(rec)

Successfully loaded model
                                          recipe               score  \
0                                  szechuan fish  0.9242935430919742   
1     tod mun goong   thai fish and shrimp cakes  0.9213254375755039   
2  copycat p f  chang s singapore street noodles  0.9169091062273881   
3   thai chicken and eggplant  aubergine   curry  0.9158816618127215   
4                                imperial fondue    0.91463966647726   

                                         ingredients  \
0  red snapper,green onions,garlic,fish stock,soy...   
1  flounder,shrimp,onion,cilantro,green beans,cor...   
2  rice noodles,oil,shrimp,boneless skinless chic...   
3  unsweetened coconut milk,green curry paste,bon...   
4  boneless skinless chicken,white fish fillet,la...   

                                               steps  
0  ['in a wok brown the fish on all sides with th...  
1  ['process the onion and cilantro until chopped...  
2  ['boil rice stick noodles for 2 minu

In [61]:
class MeanEmbeddingVectorizer(object):

	def __init__(self, word_model):
		self.word_model = word_model
		self.vector_size = word_model.wv.vector_size

	def fit(self):  # comply with scikit-learn transformer requirement
		return self

	def transform(self, docs):  # comply with scikit-learn transformer requirement
		doc_word_vector = self.word_average_list(docs)
		return doc_word_vector

	def word_average(self, sent):
		"""
		Compute average word vector for a single doc/sentence.
		:param sent: list of sentence tokens
		:return:
			mean: float of averaging word vectors
		"""
		mean = []
		for word in sent:
			if word in self.word_model.wv.index_to_key:
				mean.append(self.word_model.wv.get_vector(word))

		if not mean:  # empty words
			# If a text is empty, return a vector of zeros.
			logging.warning("cannot compute average owing to no vector for {}".format(sent))
			return np.zeros(self.vector_size)
		else:
			mean = np.array(mean).mean(axis=0)
			return mean


	def word_average_list(self, docs):
		"""
		Compute average word vector for multiple docs, where docs had been tokenized.
		:param docs: list of sentence in list of separated tokens
		:return:
			array of average word vector in shape (len(docs),)
		"""
		return np.vstack([self.word_average(sent) for sent in docs])

In [62]:
# encode document by averaging word embeddings

# load model 
loaded_model = Word2Vec.load('models/model_cbow.bin')
if loaded_model:
    print("Successfully loaded model")

mean_vec_tr = MeanEmbeddingVectorizer(loaded_model)
doc_vec = mean_vec_tr.transform(corpus)

Successfully loaded model
